In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [3]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [4]:
feature_list

{'SYN': ['Src Port',
  'Protocol',
  'Flow Duration',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Std',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Min',
  'Bwd Pkt Len Std',
  'Flow IAT Min',
  'Fwd IAT Tot',
  'Fwd IAT Std',
  'Bwd IAT Tot',
  'Bwd IAT Mean',
  'Bwd IAT Max',
  'Bwd IAT Min',
  'Fwd Header Len',
  'Fwd Pkts/s',
  'Pkt Len Min',
  'Pkt Len Mean',
  'Pkt Len Std',
  'Pkt Len Var',
  'SYN Flag Cnt',
  'PSH Flag Cnt',
  'ACK Flag Cnt',
  'Pkt Size Avg',
  'Fwd Seg Size Avg',
  'Bwd Seg Size Avg',
  'Subflow Fwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Bwd Pkts',
  'Subflow Bwd Byts',
  'Active Mean',
  'Active Std',
  'Active Max',
  'Idle Std',
  'Idle Max',
  'Label'],
 'HTTP': ['Dst Port',
  'Flow Duration',
  'Flow IAT Max',
  'Fwd IAT Tot',
  'FIN Flag Cnt',
  'SYN Flag Cnt',
  'Label'],
 'ACK': ['TotLen Fwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Std',
  'Bwd Pkt Len Max',
  'Flow Byts/s',
  'Bwd IAT Std',
  'Pkt L

In [10]:
file_list={'SYN': ['./csvs/dos-synflooding-1-dec.pcap_Flow.csv',
  './INPUT/VAL/VAL-SYN.csv'],
 'HTTP': ['./csvs/mirai-httpflooding-4-dec.pcap_Flow.csv',
  './INPUT/VAL/VAL-HTTP.csv'],
 'ACK': ['./csvs/mirai-ackflooding-4-dec.pcap_Flow.csv',
  './INPUT/VAL/VAL-ACK.csv'],
 'UDP': ['./csvs/mirai-udpflooding-4-dec.pcap_Flow.csv',
  './INPUT/VAL/VAL-UDP.csv'],
 'ARP': ['./csvs/mitm-arpspoofing-6-dec.pcap_Flow.csv',
  './INPUT/VAL/VAL-ARP.csv'],
 'SP': ['./csvs/scan-hostport-3-dec.pcap_Flow.csv', './INPUT/VAL/VAL-SP.csv'],
 'BF': ['./csvs/mirai-hostbruteforce-5-dec.pcap_Flow.csv',
  './INPUT/VAL/VAL-BF.csv']}

In [6]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

# RandomizedSearchCV  DT

In [8]:
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.764    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:54<00:00,  1.09s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.235    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  6.71it/s]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.8950041883779875   0.149    24      


...

SP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.126    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.49it/s]


BF
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9980172762937576   0.227    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.04it/s]


    criterion      max_depth    max_features    min_samples_split        F1          Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  -----------  ------  ----  --------
 0  gini                   5              35                    6  1         0             1.159     2  SYN
 1  entropy               16               2                    2  1         0             0.141    48  HTTP
 2  gini                   5               2                    8  0.895004  0             0.068    16  ACK
 3  entropy                3              11                    2  1         0             0.079    27  UDP
 4  entropy               28              27                    2  0.919672  0             0.198     4  ARP
 5  gini                   4              16                    9  1         0             0.088    28  SP
 6  gini                  19              10                    3  0.998017  1.11022e-16   0.15     24  BF


# RandomizedSearchCV RF

In [9]:
lines=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]



for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))



    # use a full grid over all parameters
    param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                  "n_estimators" : sp_randint(1, 200),
                  "max_features": sp_randint(1, 11),
                  "min_samples_split":sp_randint(2, 11),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    second=time()
    f1=[]
    clf=RandomForestClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("RF_HPO.csv",index=False)

final_parametres=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.236    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:58<00:00, 23.83s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.312    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:03<00:00,  6.33s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.8950041883779871   0.203    0       


...

SP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.189    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.72s/it]


BF
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9980172762937574   0.267    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:04<00:00,  6.47s/it]

    bootst    criter      max_depth    max_features    min_samp_split    n_estimators        F1          Std    Time    No  Attack
--  --------  --------  -----------  --------------  ----------------  --------------  --------  -----------  ------  ----  --------
 0  False     gini                6               5                 9              73  1         0            16.646     5  SYN
 1  False     entropy            14               9                 2               7  1         0             3.232     1  HTTP
 2  False     entropy             4               9                 7             183  0.895004  0             4.091     8  ACK
 3  False     gini                6               5                10               7  1         0             1.698     7  UDP
 4  False     entropy            25               1                 3              48  0.919637  7.08033e-05   2.399     1  ARP
 5  False     entropy             6               5                 6              29  1       

# XGBClassifier

In [11]:
import xgboost

lines=[['n_estimators', 'min_child_weight', 'max_depth','learning_rate', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    


    param_grid =   {
    'n_estimators': [100, 500, 900, 1100, 1500],
    'max_depth': [2, 3, 5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.15, 0.20],
    'min_child_weight': [1, 2, 3, 4]
    }

    second=time()
    f1=[]
    clf=xgboost.XGBClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(5)):
        second=time()
        a,b,clf=run_random_search(xgboost.XGBClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("XGB_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
default                             1.0                  3.633    0       


 20%|████████████████▌                                                                  | 1/5 [04:26<17:44, 266.11s/it]

{'n_estimators': 500, 'min_child_weight': 2, 'max_depth': 3, 'learning_rate': 0.05}        1.0                  266.107  0       


 40%|█████████████████████████████████▏                                                 | 2/5 [07:12<10:22, 207.46s/it]

{'n_estimators': 1500, 'min_child_weight': 2, 'max_depth': 10, 'learning_rate': 0.05}      1.0                  166.404  1       


...

{'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 15, 'learning_rate': 0.05}      0.9960235354972198   37.193   2       


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:37<00:25, 25.02s/it]

{'n_estimators': 900, 'min_child_weight': 2, 'max_depth': 10, 'learning_rate': 0.1}        0.9960235354972198   20.109   3       


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:56<00:00, 23.22s/it]

{'n_estimators': 900, 'min_child_weight': 2, 'max_depth': 15, 'learning_rate': 0.15}       0.9960235354972198   18.807   4       
      n_estimators    min_child_weight    max_depth    learning_rate        F1          Std     Time    No  Attack
--  --------------  ------------------  -----------  ---------------  --------  -----------  -------  ----  --------
 0             500                   2            3             0.05  1         0            266.107     0  SYN
 1            1500                   2           10             0.05  1         0            166.404     1  SYN
 2             900                   1           15             0.2   1         0             98.541     2  SYN
 3            1500                   1           10             0.1   1         0            119.483     3  SYN
 4             100                   1            2             0.15  1         0             55.826     4  SYN
 5            1500                   1            2             0.2   0.99943 

In [15]:
results[-7:]

,bootst,criter,max_depth,max_features,min_samp_split,n_estimators,F1,Std,Time,No,Attack
7,100,1,2,0.15,1.000000,0.000000e+00,55.826,4.0,SYN,NaN,None
8,1500,1,2,0.20,0.999430,1.110223e-16,29.358,0.0,HTTP,NaN,None
9,1100,1,3,0.10,0.894197,0.000000e+00,18.165,3.0,ACK,NaN,None
10,500,4,2,0.05,1.000000,0.000000e+00,10.858,1.0,UDP,NaN,None
11,1100,3,3,0.20,0.919495,0.000000e+00,36.191,1.0,ARP,NaN,None
12,1500,1,2,0.10,1.000000,0.000000e+00,153.923,1.0,SP,NaN,None
13,900,2,10,0.15,0.996024,0.000000e+00,23.954,1.0,BF,NaN,None


In [13]:
final_parametres=[['n_estimators', 'max_depth', 'learning_rate','min_child_weight', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

IndexError: list index out of range

# RandomizedSearchCV  KNeighborsClassifier

In [16]:
lines=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    

    # use a full grid over all parameters
    param_grid = {"n_neighbors" : sp_randint(1,64) ,  
                 "leaf_size": sp_randint(1,50) , 
                  "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"],
                  "weights" : ["uniform", "distance"]}
    second=time()
    f1=[]
    clf=KNeighborsClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(KNeighborsClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("knn_HPO.csv",index=False)

final_parametres=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["n_neighbors"].min()
    df=df[df["n_neighbors"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    

SYN
default                             0.9614496200317089   56.248   24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [09:13<00:00, 55.40s/it]


HTTP
default                             0.9170290653692525   1.984    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.03s/it]


ACK
default                             0.3703133025166922   2.232    24      


...

SP
default                             0.9571576691781299   2.607    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.39s/it]


BF
default                             0.8150577654437577   1.333    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.36it/s]

    algorithm      leaf_size    n_neighbors  weights          F1    Std    Time    No  Attack
--  -----------  -----------  -------------  ---------  --------  -----  ------  ----  --------
 0  auto                  42              6  distance   1             0  47.95      4  SYN
 1  brute                  7              4  distance   1             0   1.511     5  HTTP
 2  auto                  11              4  distance   0.370959      0   1.038     5  ACK
 3  auto                  23              4  distance   1             0   0.487     6  UDP
 4  ball_tree              5             16  distance   0.919672      0   0.428     6  ARP
 5  kd_tree               33              3  distance   1             0   1.163     2  SP
 6  kd_tree               38              2  distance   0.998017      0   0.533     2  BF


In [17]:
from sklearn.naive_bayes import GaussianNB

In [18]:
lines=[['var_smoothing', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
        'var_smoothing': np.logspace(0,-9, num=100),
    }

 
    second=time()
    f1=[]
    clf=GaussianNB()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(GaussianNB(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("NB_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.7265716823629493   0.047    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.49it/s]


HTTP
default                             0.8041507420497844   0.0      0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.98it/s]


ACK
default                             0.7992571884984025   0.006    0       


...

SP
default                             0.9721473252804143   0.007    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.58it/s]


BF
default                             0.27946484729619614  0.0      0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 16.21it/s]

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      2.84804e-06  0.747152  0             0.67      0  SYN
 1      2.84804e-06  0.747152  0             0.664     1  SYN
 2      1.51991e-06  0.747152  0             0.675     2  SYN
 3      1.87382e-06  0.747152  0             0.659     3  SYN
 4      3.51119e-07  0.741789  0             0.663     4  SYN
 5      3.51119e-06  0.747152  0             0.67      5  SYN
 6      2.31013e-06  0.747152  0             0.677     6  SYN
 7      3.51119e-06  0.747152  0             0.679     7  SYN
 8      1.87382e-05  0.741609  0             0.667     8  SYN
 9      1e-05        0.747152  0             0.669     9  SYN
10      2.84804e-09  0.804151  0             0.066     0  HTTP
11      1.23285e-08  0.752592  0             0.063     1  HTTP
12      3.51119e-09  0.804151  0             0.086     2  HTTP
13      1e-09        0.804151  0             0.063     3  H

In [19]:

final_parametres=[["var_smoothing","F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      2.84804e-06  0.747152  0             0.67      0  SYN
 1      2.84804e-09  0.804151  0             0.066     0  HTTP
 2      1e-09        0.799257  0             0.052     4  ACK
 3      1e-09        0.627066  0             0.056     5  UDP
 4      1e-09        0.463065  0             0.064     0  ARP
 5      2.31013e-09  0.972147  1.11022e-16   0.07      0  SP
 6      0.1          0.504825  0             0.062     2  BF


# LR

In [20]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
     'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' : ['none', 'l1', 'l2', 'elasticnet'],
    'C' : loguniform(1e-5, 100)
        }
    

 
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(LogisticRegression(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

SYN
default                             0.9877752607048639   0.444    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:16<00:00,  7.66s/it]


HTTP
default                             0.3436137821238856   0.058    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.66it/s]


ACK
default                             0.894196863115009    0.065    0       


...

SP
default                             0.9622386313788566   0.224    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.14s/it]


BF
default                             0.45874587458745875  0.064    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.26s/it]

               C  penalty    Solver           F1          Std    Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  ------  ----  --------
 0   3.12528      none       lbfgs      0.987775  1.11022e-16  21.657     0  SYN
 1   0.296439     none       lbfgs      0.987775  1.11022e-16   6.966     1  SYN
 2   1.54609e-05  l2         liblinear  0.974941  1.11022e-16   5.812     2  SYN
 3   0.000636388  none       lbfgs      0.987775  1.11022e-16   5.227     3  SYN
 4   0.00103276   l2         lbfgs      0.987775  1.11022e-16   4.564     4  SYN
 5  15.7231       none       lbfgs      0.987775  1.11022e-16   7.806     5  SYN
 6   0.00686268   none       lbfgs      0.987775  1.11022e-16   7.071     6  SYN
 7   0.00870369   none       lbfgs      0.987775  1.11022e-16   3.903     7  SYN
 8  15.8866       none       lbfgs      0.987775  1.11022e-16   8.116     8  SYN
 9   0.11427      l2         lbfgs      0.987775  1.11022e-16   5.421     9  SYN
10   0.237316     l2

# MLP

In [11]:
import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}


In [12]:
# MLP

import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}




lines=[['solver', 'learning_rate','hidden_layer_sizes','alpha','activation', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    

    param_grid =  {
        'hidden_layer_sizes': layers,
        'solver': ['sgd', 'adam', 'lbfgs'],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001,0.001,0.01,0.1, 0.05],
        'learning_rate': ['constant','adaptive']
    }



    second=time()
    f1=[]
    clf=MLPClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(MLPClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("ann_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.9682803935982738   2.047    0       


100%|████████████████████████████████████████████| 1/1 [07:50<00:00, 470.97s/it]


{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (901,), 'alpha': 0.1, 'activation': 'relu'} 0.9877752607048639   470.964  0       
HTTP
default                             0.28181108508977365  0.145    0       


100%|█████████████████████████████████████████████| 1/1 [01:23<00:00, 83.00s/it]

{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (408,), 'alpha': 0.001, 'activation': 'tanh'} 0.7605141444027561   83.002   0       
ACK


...

default                             0.9509566538479798   0.655    0       


100%|████████████████████████████████████████████| 1/1 [02:09<00:00, 129.31s/it]


{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (1533,), 'alpha': 0.05, 'activation': 'relu'} 0.9379884981570639   129.306  0       
BF
default                             0.49926120304347554  0.097    0       


100%|████████████████████████████████████████████| 1/1 [01:42<00:00, 102.55s/it]

{'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (892, 1585), 'alpha': 0.001, 'activation': 'tanh'} 0.7317536041939712   102.547  0       
    solver    learning_rate    hidden_layer_sizes      alpha  activation          F1    Std     Time    No  Attack
--  --------  ---------------  --------------------  -------  ------------  --------  -----  -------  ----  --------
 0  adam      constant         (901,)                 0.1     relu          0.987775      0  470.964     0  SYN
 1  adam      constant         (408,)                 0.001   tanh          0.760514      0   83.002     0  HTTP
 2  adam      adaptive         (95,)                  0.01    relu          0.895004      0   68.525     0  ACK
 3  lbfgs     constant         (1043,)                0.0001  tanh          0.934167      0  201.501     0  UDP
 4  sgd       adaptive         (1637,)                0.0001  tanh          0.734597      0   56.965     0  ARP
 5  adam      constant         (1533,)         

-------------

# RandomizedSearchCV  SVM

In [13]:
lines=[[ 'gamma','C', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    


    param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}  

    second=time()
    f1=[]
    clf=svm.SVC()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("svm_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.7754773169825647   0.505    0       


100%|████████████████████████████████████████████| 1/1 [04:52<00:00, 292.94s/it]

{'gamma': 0.001, 'C': 1}                                                                   1.0                  292.943  0       
HTTP


default                             0.583765772216309    0.181    0       


100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.27s/it]


...

{'gamma': 0.01, 'C': 10}                                                                   0.9194952962115434   0.26     0       
SP
default                             0.7494355816773952   0.018    0       


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.65it/s]


{'gamma': 0.01, 'C': 10}                                                                   1.0                  0.273    0       
BF
default                             0.4857314680908948   0.036    0       


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.19it/s]

{'gamma': 1, 'C': 10}                                                                      0.9980172762937574   0.843    0       
      gamma    C        F1    Std     Time    No  Attack
--  -------  ---  --------  -----  -------  ----  --------
 0    0.001    1  1             0  292.943     0  SYN
 1    0.1      1  0.99886       0    3.267     0  HTTP
 2    0.1     10  0.895004      0    1.167     0  ACK
 3    1       10  1             0    0.466     0  UDP
 4    0.01    10  0.919495      0    0.26      0  ARP
 5    0.01    10  1             0    0.274     0  SP
 6    1       10  0.998017      0    0.843     0  BF
